In [83]:
# Решето Эратосфена
def generate_prime_table(n):
    multiples = set()
    for i in range(2, n+1):
        if i not in multiples:
            yield i
            multiples.update(range(i*i, n+1, i))

# subexponentional funtion
def L(c, g, q): return e^(c*(log(q))^g *(log(log(q)))^(1-g))

In [86]:
def check_factor(number, primeTable):
    # we should not use only factor function
    # 1. we should try Miller-Rabin test [TODO]
    # 2. we should try Brenet's algoritm (or its analog like Lenstra alg) [TODO]
    #-----------------------------------
    decomposition = factor(number)
    # ----------------------------------
    numberPrimeFactors = set(map(lambda x: x[0], decomposition))
    success = (numberPrimeFactors.issubset(primeTable))
    
    return (success, decomposition)
        

def generate_equasion(a, q, primeTable):
    equasionGenerated = False
    decomposition = []
    
    while not equasionGenerated:
        xsi = randint(2, q-1)
        axsi = ring(a)^xsi
        equasionGenerated, decomposition = check_factor(axsi, primeTable)
        
    coefficients = list(map(lambda x: x[1] if x[1] in primeTable else 0, decomposition))
    
    return (xsi, coefficients)

In [85]:
def adleman_log(a, b, p): # return log_a (b) (mod p)
    if (p not in Primes()):
        return None
    
    ring = IntegerModRing(p)
    q = ring(a).multiplicative_order()
    border = ceil(L(1/2, 1/2, q)) #it is taken like this because of complexity balance
    primeTable = set(generate_prime_table(border))
    
    # generating the equasions
    matrixGenerated = False
    remainedEquations = len(primeTable)
    equasions = []
    while not matrixGenerated: 
        equasions += [generate_equasion(a,q,primeTable) for _ in range(remainedEquations)]
        coefficients = list(map(lambda x: x[1], equasions))

        linearIndependentColumns = Matrix(ring, coefficients).pivots()
        
        remainedEquations = len(primeTable) - len(linearIndependentColumns)
        matrixGenerated = (remainedEquations == 0)
        
        equasions = [ equasions[i] for i in linearIndependentColumns]
        
    
    # at this point equasions are generated, so we're solving them
    y = vector(list(map(lambda x: x[0], equasions)))
    A = Matrix(ring, list(map(lambda x: x[1], equasions)))
    x = (A^-1) * y
    
    # finding log_a (b)
    while True:
        xsi = randint(2, q-1)
        bxsi = ring(b)^xsi
        found, decomposition = check_factor(bxsi, primeTable)
        if found:
            b_coeffisients = vector(list(map(lambda x: x[1] if x[1] in primeTable else 0, decomposition)))
            
            return (ring(xsi)^-1)*(x.dot_product(b_coeffisients))